In [1]:
# TODO modify to fix the structure of the proj
# import lib
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.model_selection import train_test_split

import os
import random

import config

In [2]:
# setting seed
def settle_seed(seed= 42):
  os.environ['PYTHONHASHSEED'] = str(seed)
  np.random.seed(seed)
  random.seed(seed)
  torch.manual_seed(seed) 
#   torch.cuda.manual_seed(seed) 
#   torch.cuda.manual_seed_all(seed) 
#   torch.backends.cudnn.deterministic = True 
#   torch.backends.cudnn.benchmark = False 
#   torch.backends.cudnn.enabled = True 
settle_seed()

In [3]:
# define data and number of classes in data
dataset = config.train_dataset_csv_path
model_save_path = config.train_model_output_path
NUM_CLASSES = 4

In [4]:
# setting dataloader
X_dataset = np.loadtxt(dataset, delimiter=',', dtype='float32', usecols=list(range(1, (21 * 2) + 1)))
y_dataset = np.loadtxt(dataset, delimiter=',', dtype='int32', usecols=(0))
class KeyPointDataset(Dataset):
    def __init__(self,X_dataset, y_dataset):
        self.X = X_dataset
        self.y = y_dataset
    def __getitem__(self, item):
        return torch.tensor(self.X[item],dtype= torch.float32), torch.tensor(self.y[item],dtype=torch.long) 
    def __len__(self):
        return len(self.X)
train_dataset = KeyPointDataset(X_dataset, y_dataset)
train_dataset, valid_dataset = torch.utils.data.random_split(train_dataset,[3200, 800])

train_data_loader = DataLoader(train_dataset, batch_size= 32, shuffle= True )
valid_data_loader = DataLoader(valid_dataset, batch_size= 32, shuffle= True )

In [6]:
# define model and parameters of training
model = nn.Sequential(
    nn.Dropout(0.2),
    nn.Linear(21*2, 128),
    nn.ReLU(),
    # nn.Sigmoid(),
    nn.Dropout(0.4),
    nn.Linear(128, 32),
    nn.ReLU(),
    # nn.Sigmoid(),
    nn.Linear(32, NUM_CLASSES),
    nn.Softmax(dim=1)
)

criterion = nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(model.parameters(),lr = 0.001,weight_decay= 1e-4)
epochs = 20
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
def evaluate(step):

    model.eval()
    valid_loss = [] 
    valid_acc = [] 
    for eval_batch in valid_data_loader:
        X, label = eval_batch
        X = X.to(device)
        label = label.to(device) 
        pred = model.forward(X) 
        loss = criterion(pred,label) 
        acc = sum(torch.argmax(pred,dim = 1) == label) / len(label) 
        valid_loss.append(loss) 
        valid_acc.append(acc)
        # tb.save_value('loss', 'loss', loss.item(), step)
        # tb.save_value('acc', 'acc', acc.item(), step)
        # valid_writer.add_scalar('loss',loss.item(),step) 
        # valid_writer.add_scalar('acc',acc.item(),step) 
    valid_loss = sum(valid_loss) / len(valid_loss) 
    valid_acc = sum(valid_acc) / len(valid_acc) 
    return valid_loss,valid_acc
def train():
    step = 0 
    model.to(device) 
    min_loss = float('inf') 
    for epoch in range(epochs):
        model.train() 
        train_loss = [] 
        train_acc = []
        for batch in train_data_loader:
            optimizer.zero_grad() 
            X, label = batch 
            # X = X.unsqueeze(1) # in channels 
            X = X.to(device) 
            label = label.to(device)
            pred = model.forward(X) 
            loss = criterion(pred,label) 
            acc = sum(torch.argmax(pred, dim = 1) == label) / len(label) 
            train_loss.append(loss) 
            train_acc.append(acc)
            # tb.save_value('Train Loss', 'train_loss', loss.item(), step)
            # tb.save_value('Train acc', 'train_acc', acc.item(), step)
            # train_writer.add_scalar('loss',loss.item(),step) 
            # train_writer.add_scalar('acc',acc.item(),step) 
            loss.backward()
            optimizer.step()
            step += 1
            if step % 500 == 0:
                valid_loss,valid_acc = evaluate(step)
                print('Validing: step:%d,mean_loss:%.5f,mean_acc:%.5f' % ( step,valid_loss,valid_acc)) 
                if valid_loss < min_loss:
                    min_loss = valid_loss 
                    torch.save(model.state_dict(), model_save_path) 
        mean_loss = sum(train_loss) / len(train_loss) 
        mean_acc = sum(train_acc) / len(train_acc) 
        print('Trainging: epoch:%d,step:%d,mean_loss:%.5f,mean_acc:%.5f' % ( epoch,step,mean_loss,mean_acc))
train()

Trainging: epoch:0,step:100,mean_loss:1.19310,mean_acc:0.63031
Trainging: epoch:1,step:200,mean_loss:0.87765,mean_acc:0.90625
Trainging: epoch:2,step:300,mean_loss:0.84239,mean_acc:0.91781
Trainging: epoch:3,step:400,mean_loss:0.82738,mean_acc:0.92563
Validing: step:500,mean_loss:0.77394,mean_acc:0.97500
Trainging: epoch:4,step:500,mean_loss:0.81329,mean_acc:0.94000
Trainging: epoch:5,step:600,mean_loss:0.81033,mean_acc:0.94000
Trainging: epoch:6,step:700,mean_loss:0.81111,mean_acc:0.93875
Trainging: epoch:7,step:800,mean_loss:0.80462,mean_acc:0.94437
Trainging: epoch:8,step:900,mean_loss:0.79890,mean_acc:0.94719
Validing: step:1000,mean_loss:0.76817,mean_acc:0.97875
Trainging: epoch:9,step:1000,mean_loss:0.80083,mean_acc:0.94781
Trainging: epoch:10,step:1100,mean_loss:0.79973,mean_acc:0.94281
Trainging: epoch:11,step:1200,mean_loss:0.79672,mean_acc:0.95094
Trainging: epoch:12,step:1300,mean_loss:0.79435,mean_acc:0.95281
Trainging: epoch:13,step:1400,mean_loss:0.80020,mean_acc:0.94219
